# Daily Results

## Initialisation

Basic approach to determine the project directory

In [1]:
import os
import sys
import glob

from datetime import datetime
import time

import csv
import json

In [2]:
projdir = os.path.realpath(os.path.join(sys.path[0], '..'))

In [3]:
CONFIG_DIR = 'config'
SESSIONS_DIR = 'sessions'
GPSDATA_DIR = 'gpsdata'

In [4]:
CONFIG_JSON = 'config.json'
ENTRANTS_JSON = 'entrants.json'
SESSIONS_JSON = 'sessions.json'

In [5]:
DEFAULT_NATIONALITY = 'England'

In [6]:
try:
    wsw_full_refresh = int(os.environ['WSW_FULL_REFRESH'])
except:
    wsw_full_refresh = 1
    
verbose = True

## Generic Class

Generic class to ensure that all custom classes are printable

In [7]:
class Printable:
    def __repr__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)
    
    def logInfo(self, msg):
        print('INFO:', msg)

    def logWarning(self, msg):
        print('WARNING:', msg)

    def logError(self, msg):
        print('ERROR:', msg)

## SpeedRun Class

Class to manage runs by an entrant

In [8]:
class SpeedRun(Printable):
    def __init__(self, courseId, startTime, duration, speed, cog):
        '''Initialise speed run object'''    
        
        self.courseId, self.startTime, self.duration, self.speed, self.cog = courseId, startTime, duration, speed, cog

## Entrant Class

Class to manage entrants

In [9]:
class Entrant(Printable):
    def __init__(self, entrantId, entrantDict, iswcRequired=False):
        '''Initialise entrant object'''

        # Entrant ID is generated by the calling routine
        self.entrantId = entrantId
        
        # Retain original dictionary for later use by reports and filters
        self.entrantDict = entrantDict
        
        # Quickly check if entrant details as defined in the config
        for key in config['Entrants']:
            if key != 'ISWC Member' or iswcRequired:
                # Check the attribute is present
                if key not in entrantDict:
                    raise ValueError('Missing "{}" for entrant #{}'.format(key, entrantId))
                # Check the attribute is valid
                if entrantDict[key] not in config['Entrants'][key]:
                    raise ValueError('Unexpected "{}" for entrant #{} - {}'.format(key, entrantId, entrantDict[key]))

        # Record personal details
        self.firstName = entrantDict['First Name']
        self.familyName = entrantDict['Family Name']
        self.gender = entrantDict['Gender']

        # Record nationality
        try:
            self.nationality = entrantDict['Nationality']
        except:
            self.nationality = DEFAULT_NATIONALITY

        # Record fleet details
        self.craftType = entrantDict['Craft Type']
        self.status = entrantDict['Status']
        self.entryType = entrantDict['EntryType']
        self.firstTimer = entrantDict['First Timer']

        # Record GPS details
        self.gt31SerialNumbers = set()
        if 'GT31 SN' in entrantDict:
            for gt31SerialNumber in str(entrantDict['GT31 SN']).split(','):
                if gt31SerialNumber != '':
                    self.gt31SerialNumbers.add(gt31SerialNumber)

        # Record membership details
        self.ukwaMember = entrantDict['UKWA Member']
        if iswcRequired:
            self.iswcMember = entrantDict['ISWC Member']

        # Note: A session is a complete day in WSW terms but may comprise of multiple courses
        self.sessions = {}


    def storeRun(self, courseDate, courseId, startTime, duration, speed, cog):
        '''Store run in memory'''
        
        speedRun = SpeedRun(courseId, startTime, duration, speed, cog)

        if courseDate not in self.sessions:
            self.sessions[courseDate] = []
            
        self.sessions[courseDate].append(speedRun)

## Course Class

Class to manage courses - start / end times

In [10]:
class Course(Printable):
    def __init__(self, courseDate, courseId, courseDict):
        '''Initialise course object'''

        self.courseDate = courseDate
        self.courseId = courseId
        self.startTime = courseDict['Start Time']
        self.endTime = courseDict['End Time']
        
        self.gt31Ids = set()
        self.speeds = {}

        self.minStartTime = '23:59:59'
        self.maxStartTime = '00:00:00'
        self.numRuns = 0
        self.numGt31s = 0


    def getEntrant(self, gt31Ids, gt31Id, gt31Serial):
        '''Get entrant from the GT-31 ID and serial'''

        if gt31Id not in gt31Ids:
            self.logWarning('Unrecognised GT-31 ID on {} - {}'.format(self.courseDate, gt31Id))

            entrant = None
        else:
            if gt31Serial not in gt31Ids[gt31Id].gt31SerialNumbers:
                self.logWarning('Unrecognised GT-31 SN for {} {} ({}) on {} - {} vs {}'.format(
                        gt31Ids[gt31Id].firstName, gt31Ids[gt31Id].familyName, gt31Ids[gt31Id].craftType,
                        self.courseDate,
                        gt31Serial, gt31Ids[gt31Id].gt31SerialNumbers))

            self.gt31Ids.add(gt31Id)

            entrant = gt31Ids[gt31Id]
            
        return entrant


    def storeRun(self, entrant, startTime, duration, speed, cog):
        '''Store run in memory'''

        if startTime < self.minStartTime:
            self.minStartTime = startTime
        if startTime > self.maxStartTime:
            self.maxStartTime = startTime

        if startTime >= self.startTime and startTime <= self.endTime:
            entrant.storeRun(self.courseDate, self.courseId, startTime, duration, speed, cog)

        self.numRuns += 1


    def loadRuns(self, csvPath, gt31Ids):
        '''Read speeds from CSV files'''

        prevGt31Id = None

        with open(csvPath, 'r') as f:
            csvReader = csv.reader(f)
            for values in csvReader:
                # GPSResults does not output a header line
                run, filename, startTime, duration, speed, cog = values

                # Split up record and format the GT-31 details
                gt31Id, gt31Serial, fileDate, fileTime = os.path.splitext(filename)[0].split('_')[:4]
                gt31Id = gt31Id.upper()

                # Validate each newly encountered GT-31
                if gt31Id != prevGt31Id:
                    entrant = self.getEntrant(gt31Ids, gt31Id, gt31Serial)

                # Only store runs for recognised entrants (via GT-31 ID)
                if entrant:
                    self.storeRun(entrant, startTime, duration, speed, cog)
                
                prevGt31Id = gt31Id

            self.logInfo('{} runs from {} recognised GT-31s for course {} on {} - {} to {}'.format(
                    self.numRuns, len(self.gt31Ids), self.courseId, self.courseDate, self.minStartTime, self.maxStartTime))
            
        if self.minStartTime < self.startTime:
            self.logWarning('Runs found before course {} opened on {} - earliest was {}'.format(
                self.courseId, self.courseDate, self.minStartTime))

        if self.maxStartTime > self.endTime:
            self.logWarning('Runs found after course {} closed on {} - latest was {}'.format(
                self.courseId, self.courseDate, self.maxStartTime))

## Session Class

Class to manage sessions

In [11]:
class Session(Printable):
    def __init__(self, configPath, dataPath):
        '''Initialise session object'''

        self.configPath = configPath
        self.dataPath = dataPath

        self.date = os.path.basename(configPath)
        self.year = self.date[:4]

        self.courses = {}


    def loadConfig(self):
        '''Read config from JSON'''

        filename = os.path.join(self.configPath, SESSIONS_JSON)
        with open(filename, 'r', encoding='utf-8') as f:
            jsonTxt = f.read()
            courseDict = json.loads(jsonTxt)
            
            for courseId in courseDict:
                # Course ID should be uppercase but just to be sure...
                courseId = courseId.upper()
                if courseId in self.courses:
                    raise ValueError('Duplicate course "{}" for {}'.format(courseId, self.date))
                course = Course(self.date, courseId, courseDict[courseId])
                self.courses[courseId] = course


    def loadRuns(self, gt31Ids):
        '''Read speeds from CSV files'''

        csvPaths = sorted(glob.glob(os.path.join(self.dataPath, '*')))
        
        for csvPath in csvPaths:
            prefix, sessionDate, courseId = os.path.splitext(os.path.basename(csvPath))[0].split('_')
            suffix = os.path.splitext(csvPath)[1]

            # Verify the filename
            if suffix.lower() != '.csv':
                raise ValueError('Invalid file suffix "{}" for {}'.format(suffix, os.path.basename(csvPath)))
            if prefix.upper() != 'GPSDATA':
                raise ValueError('Invalid file prefix "{}" for {}'.format(prefix, os.path.basename(csvPath)))
            if sessionDate != self.date:
                raise ValueError('Invalid file date "{}" for {}'.format(sessionDate, os.path.basename(csvPath)))
            if courseId.upper() not in self.courses:
                raise ValueError('Invalid course "{}" for {}'.format(courseId, os.path.basename(csvPath)))
            
            course = self.courses[courseId.upper()]
            course.loadRuns(csvPath, gt31Ids)

## Event Class

Class to manage events

In [12]:
class Event():
    def __init__(self, path):
        self.path = path
        self.year = int(os.path.basename(path))

        if self.year in config['Series']['ISWC']['Years']:
            self.iswcYear = True
        else:
            self.iswcYear = False

        self.entrants = {}
        self.gt31Ids = {}
        self.gt31Serials = {}
        
        self.sessions = {}


    def processEvent(self):
        '''Read entrants from JSON'''

        if verbose:
            print('Processing {}...'.format(self.year))

        self.loadEntrants()  
        self.indexGt31s()
        self.loadSessions()

        if verbose:
            print('All done!\n')


    def loadEntrants(self):
        '''Read entrants from JSON'''

        filename = os.path.join(self.path, CONFIG_DIR, ENTRANTS_JSON)
        with open(filename, 'r', encoding='utf-8') as f:
            jsonTxt = f.read()
            entrantDicts = json.loads(jsonTxt)
            
            for entrantDict in entrantDicts:
                entrantId = len(self.entrants)
                entrant = Entrant(entrantId, entrantDict, iswcRequired=self.iswcYear)
                self.entrants[entrantId] = entrant

        if verbose:
            print('{} entrants loaded'.format(len(self.entrants)))


    def indexGt31s(self):
        '''Create GT-31 indices for entrants'''

        for entrantId in self.entrants.keys():
            entrant = self.entrants[entrantId]

            if 'GT31 ID' in entrant.entrantDict:
                self.gt31Ids[entrant.entrantDict['GT31 ID'].upper()] = entrant
            if 'GT31 SN' in entrant.entrantDict:
                self.gt31Serials[entrant.entrantDict['GT31 SN']] = entrant
        

    def summariseEntrants(self):
        '''Print summary of entrants'''
        
        

    def loadSessions(self):
        '''Load all of the event sessions - absence of config will result in an exception (intentional)'''
        
        dataPaths = sorted(glob.glob(os.path.join(self.path, GPSDATA_DIR, '20[0-9][0-9][0-1][0-9][0-3][0-9]')))
        
        for dataPath in dataPaths:
            configPath = os.path.join(self.path, SESSIONS_DIR, os.path.basename(dataPath))
            session = Session(configPath, dataPath)

            session.loadConfig()
            session.loadRuns(self.gt31Ids)

            self.sessions[session.date] = session

## Process Years

Process all available years

In [13]:
pc1 = time.perf_counter()

In [14]:
# Read main config
filename = os.path.join(projdir, CONFIG_DIR, CONFIG_JSON)
with open(filename, 'r', encoding='utf-8') as f:
    jsonTxt = f.read()
    config = json.loads(jsonTxt)

In [15]:
# Only process the current year (for now)
eventPaths = sorted(glob.glob(os.path.join(projdir, '20[0-9][0-9]')))
year = datetime.now().year
for eventPath in eventPaths:
    if int(os.path.basename(eventPath)) <= year:
        if int(os.path.basename(eventPath)) == year or wsw_full_refresh:
            event = Event(eventPath)
            event.processEvent()

Processing 2018...
84 entrants loaded
INFO: 67 runs from 24 recognised GT-31s for course H1 on 20181013 - 11:00:53 to 12:00:00
INFO: 636 runs from 56 recognised GT-31s for course H2 on 20181013 - 12:24:12 to 16:01:53
INFO: 209 runs from 31 recognised GT-31s for course H1 on 20181014 - 12:35:25 to 15:08:17
INFO: 405 runs from 37 recognised GT-31s for course H1 on 20181015 - 10:03:01 to 16:00:40
INFO: 494 runs from 48 recognised GT-31s for course S1 on 20181016 - 12:19:29 to 16:50:40
INFO: 444 runs from 37 recognised GT-31s for course H1 on 20181018 - 08:31:03 to 13:08:40
All done!

Processing 2019...
68 entrants loaded
INFO: 15 runs from 5 recognised GT-31s for course H1 on 20191005 - 14:16:32 to 16:07:10
INFO: 304 runs from 38 recognised GT-31s for course H1 on 20191006 - 10:01:54 to 12:30:49
INFO: 378 runs from 37 recognised GT-31s for course H2 on 20191006 - 13:26:28 to 16:00:40
INFO: 313 runs from 28 recognised GT-31s for course H1 on 20191007 - 11:48:52 to 15:59:27
INFO: 593 runs f

In [16]:
pc2 = time.perf_counter()
print("Reports completed in %0.2f seconds" % (pc2 - pc1))

Reports completed in 0.21 seconds


## All Done!